# Neural Trojan

In [8]:
import numpy as np
from tensorflow.keras.models import load_model

## Internal Neuron Selection
According to the paper "Trojan Attack on Neural Networks", choosing the last internal layer leads to poor performance, so the layer to choose the neuron(s) to attack is easy:
We will attack the first (out of two) dense layers, which has 500 neurons.

In [22]:
# Load pre-trained model
model_path = 'model-3x3.keras'
model = load_model(model_path)

In [24]:
for layer in model.layers:
    print(layer.name, layer.__class__.__name__)

conv2d_2 Conv2D
max_pooling2d_2 MaxPooling2D
flatten_2 Flatten
dense_4 Dense
dense_5 Dense


In [25]:
# Retrieve weights between the first and second dense layer
first_dense_layer = model.get_layer('dense_4')
weights, biases = first_dense_layer.get_weights()

number_of_neurons_to_attack = 10

neuron_influence = np.sum(np.abs(weights), axis=1)
most_influential_neurons = np.argsort(neuron_influence)[::-1][:number_of_neurons_to_attack]

print("Most influential neurons:", most_influential_neurons)

Most influential neurons: [4320 4328 4352 4363 4347 4800 4838 3816 3808 4358]


## Neural Trigger Generation